# Setup

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


"transformers" od Hugging Face bezpośrednio z repozytorium GitHub.

In [2]:
!pip install -q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00


Kluczowym elementem jest tutaj parametr `--no-build-isolation`. W normalnych warunkach pip tworzy izolowane środowisko do budowania pakietu, co zapewnia czystą instalację bez konfliktów z innymi bibliotekami. Jednak w przypadku flash-attn może to prowadzić do problemów, ponieważ ta biblioteka ma specyficzne zależności i wymaga dostępu do określonych wersji narzędzi kompilacyjnych oraz bibliotek systemowych.

Wyłączenie izolacji budowania pozwala flash-attn korzystać z już zainstalowanych w systemie zależności i narzędzi, co często jest niezbędne do poprawnej kompilacji tej biblioteki. Jest to szczególnie istotne w środowiskach, gdzie mamy już skonfigurowane odpowiednie wersje CUDA i innych bibliotek potrzebnych do wydajnego działania operacji na GPU.

Warto pamiętać, że użycie `--no-build-isolation` może teoretycznie prowadzić do niespójności w zależnościach, ale w przypadku flash-attn jest to często konieczny kompromis, aby zapewnić prawidłową instalację i optymalne działanie biblioteki.

In [ ]:
# Standard library imports
import os

# Third-party library imports
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

Z biblioteki `transformers` importowane są dwa kluczowe komponenty: `AutoProcessor` i `AutoModelForVision2Seq`.
- Procesor automatycznie przygotowuje dane wejściowe (w tym przypadku obrazy) do formatu wymaganego przez model.
- Model `Vision2Seq` to architektura, która przekształca obrazy (Vision) w sekwencje tekstu (2Seq) - innymi słowy, generuje tekstowe opisy obrazów.

Funkcja `load_image` z `transformers.image_utils` zapewnia wygodny sposób wczytywania obrazów w formacie odpowiednim dla modeli.

In [4]:
class CFG:
    device = 'cuda'
    model = "HuggingFaceTB/SmolVLM-Instruct"
    dtype = torch.bfloat16

In [23]:
processor = AutoProcessor.from_pretrained(CFG.model)
model = AutoModelForVision2Seq.from_pretrained(CFG.model,
                                               torch_dtype = CFG.dtype,
                                                _attn_implementation="eager" ).to('cuda')

Te dwie linie kodu przygotowują najważniejsze komponenty systemu - procesor danych i model neuronowy.

Pierwsza linia `processor = AutoProcessor.from_pretrained(CFG.model)` ładuje procesor, który jest odpowiedzialny za przygotowanie danych wejściowych. Procesor ten został wcześniej wytrenowany razem z modelem SmolVLM-Instruct i wie dokładnie, jak przekształcić surowe obrazy na format, który model potrafi zrozumieć. Obejmuje to szereg operacji, takich jak zmiana rozmiaru obrazu, normalizacja wartości pikseli czy dodanie specjalnych znaczników.

Druga linia jest bardziej złożona i ładuje sam model neuronowy z dodatkowymi parametrami:
- `AutoModelForVision2Seq.from_pretrained(CFG.model)` pobiera architekturę i wagi modelu z Hugging Face Hub
- `torch_dtype = CFG.dtype` ustawia format liczb na bfloat16, co pozwala zaoszczędzić pamięć GPU bez znaczącej straty jakości
- `_attn_implementation="eager"` określa sposób implementacji mechanizmu uwagi (attention). Tryb "eager" oznacza standardową implementację bez optymalizacji, która jest bardziej przewidywalna i łatwiejsza w debugowaniu
- `.to('cuda')` przenosi model na GPU, co jest kluczowe dla wydajności

# Model

In [ ]:
image_path1 = "/content/img1.png"

image1 = load_image(image_path1)

In [ ]:
# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Describe the image in detail"}
        ]
    },
]

In [ ]:
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(CFG.device)

W pierwszym kroku używamy `processor.apply_chat_template(messages, add_generation_prompt=True)`, który przekształca naszą strukturę wiadomości na format tekstowy zrozumiały dla modelu. Parametr `add_generation_prompt=True` dodaje specjalny znacznik, który sygnalizuje modelowi, że powinien zacząć generować odpowiedź. Jest to podobne do tego, jak w rozmowie możemy użyć pauzy lub tonu głosu, aby zasygnalizować rozmówcy, że oczekujemy jego odpowiedzi.

Drugi krok to wywołanie procesora z argumentami `text=prompt, images=[image1], return_tensors="pt"`. Procesor wykonuje tutaj kilka ważnych operacji. Tekst jest tokenizowany, czyli dzielony na mniejsze jednostki, które model potrafi przetwarzać. Obraz jest przekształcany do odpowiedniego formatu - jest skalowany do właściwego rozmiaru, normalizowany i przekształcany na tensor PyTorch (na co wskazuje parametr `return_tensors="pt"`). Jest to jak przygotowanie składników przed gotowaniem - wszystko musi być we właściwej formie i proporcjach.

Ostatni krok `inputs.to(CFG.device)` przenosi przygotowane dane na GPU. Jest to konieczne, ponieważ model znajduje się na GPU (pamiętamy o wcześniejszym `model.to('cuda')`), a dane wejściowe muszą być na tym samym urządzeniu co model. Możemy to porównać do przeniesienia składników na płytę kuchenną, gdzie będzie odbywało się gotowanie.

In [29]:
# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])

User:<image>Describe the image in detail
Assistant: The image depicts an office space with a large window looking out onto a marina and a cityscape. The window is framed by a brick wall on the left and a striped vertical blind on the right. The window is divided into three sections, with the middle section being the largest. The view outside the window shows a marina with several boats docked, and a cityscape in the distance. The cityscape includes a mix of modern and older buildings, with some trees visible in the background. The sky is overcast, with a mix of grey and light blue hues.

To the right of the window, there is a large desk with a white top. On the desk, there is a computer monitor, a keyboard, a mouse, a lamp, a small table, and a few other items. The monitor is black and has a large screen. The keyboard is black and has a standard layout. The mouse is black and has a USB port. The lamp is white and has a large bulb. The small table is white and has a few items on it, inc